# 1. Example: DataAssimBench basics and generating data from Lorenz63 model

In this notebook, we will introduce the basic principles DataAssimBench and generate data from the most simple model in the library: the Lorenz 1963 model.

### Part I: Import dabench.data

The "data" sub-package is dabench's interface for generating and loading datasets.

In [ ]:
from dabench import data

In [ ]:
# Running help on data shows that available data classes (under the heading "CLASSES")
help(data) 

In [ ]:
# We can get more info about a particular class by running help() on it
help(data.Lorenz63)

As we can see, this class implements the model first published in Lorenz, 1963:https://doi.org/10.1175/1520-0469(1963)020%3C0130:DNF%3E2.0.CO;2

This is a three-variable model. By default, our initial state (x0) is [-10.0, -15.0, 21.3].

In addition to the the initial state, there are three paramaters: sigma, rho, and beta. All of these have sensible defaults (taken from the original Lorenz 1963 paper) so there is no reason to tweak them unless we want to customize our model settings. 

### Part II: Create model object and generate data

To generate data from a dabench.data model, we first create a model object. If we want, this is where we can change the parameters and initial state of the model. But for starters, let's just stick with the defaults.

In [ ]:
l63_model = data.Lorenz63()

In [ ]:
# Generating data is as easy as using the .generate() method. This is the same for all models in dabench.data(). 
# We must specify n_steps to determine how long we want our model to run. Let's start with 1000
l63_model.generate(n_steps=1000)

### Part III: Examine generated data

The generated data for the Lorenz63 model is accessed as l63_model.values. This is the same scheme for all models/data loaders in dabench.data.

In [ ]:
# We can see how we start with the initial condition [-10.0, -15.0, 21.3] and proceed from there
print(l63_model.values)

In [ ]:
# Shape is (n_steps, system_dim), in this case (1000, 3)
print(l63_model.values.shape)

In [ ]:
# Let's plot our data using matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Since we have 3 variables in our model (X, Y, and Z), let's choose X and Z for plotting in 2D
plt.plot(l63_model.values[:,0], l63_model.values[:,2])
plt.xlabel('X')
plt.ylabel('Z')
plt.show()

Lorenz63 is famous for it's "butterfly" attractor shape. You should be able to see that in the plot above

### Part IV: Demonstrating chaos - tweaking initial conditions


The Lorenz, 1963 paper is famous for demonstrating the idea of "chaos", meaning that _minutely_ different initial conditions eventually result in wildly divergent outcomes. Let's test that here by tweaking in the intitial conditions.

In [ ]:
import numpy as np

In [ ]:
# Add 0.01 to Z in the initial state
new_x0 = l63_model.x0 + np.array([0, 0, 0.01])
print(new_x0)

In [ ]:
l63_new = data.Lorenz63(x0=new_x0)
l63_new.generate(n_steps=1000)

In [ ]:
# Now let's compare the final timesteps for our old and new model
print('Previous final values:', l63_model.values[-1])
print('New final values:', l63_new.values[-1])

We can see that we have wildly different outcomes despite a very small change to the initial conditions. This is the basic principle of chaos in weather modeling.

In [ ]:
# Despite having different outcomes, the two model trajectories have very similar overall shapes
plt.plot(l63_model.values[:,0], l63_model.values[:,2], label="Old")
plt.plot(l63_new.values[:,0], l63_new.values[:,2], label='New')
plt.xlabel('X')
plt.ylabel('Z')
plt.legend()
plt.show()

## Wrapping UP

That's it for the basics of dabench data generators! Feel free to experiment with changing the Lorenz63 parameters or the length of the run (n_steps). You can also try a different model like Lorenz96, which is a bit more complex, or go to the next example notebook for an even more complex "surface quasi-geostrophic turbulence" (sqgturb) model. 